# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df


,Unnamed: 0,city,lat,lon,max temp,humidity,cloudiness,wind speed,country,date
0,0,Kruisfontein,-34.0033,24.7314,65.88,61,4,3.40,ZA,1644342972
1,1,Ushuaia,-54.8000,-68.3000,56.82,58,40,23.02,AR,1644342951
2,2,Kharp,66.8014,65.8081,-10.43,100,100,2.91,RU,1644342972
3,3,Pevek,69.7008,170.3133,-32.08,100,73,5.26,RU,1644342973
4,4,Saint-Pierre,-21.3393,55.4781,78.48,83,0,2.30,RE,1644342973
...,...,...,...,...,...,...,...,...,...,...
258,258,Dikson,73.5069,80.5464,4.95,93,100,22.28,RU,1644343063
259,259,Serebryansk,49.6925,83.2892,9.95,94,100,2.06,KZ,1644343063
260,260,Bengkulu,-3.8004,102.2655,75.92,86,98,5.35,ID,1644343064
261,261,San Agustín Acasaguastlán,14.9500,-89.9667,73.71,85,92,3.76,GT,1644343064


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_df[["lat", "lon"]].astype(float)
humidity = cities_df["humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# creating new DF for ideal weather conditions
ideal_weather_df = cities_df.loc[(cities_df['max temp'] >= 80) &
                        (cities_df['max temp'] <= 95) &
                        (cities_df['wind speed'] < 10) &
                        (cities_df['cloudiness'] < 50) &
                        (cities_df['humidity'] < 60)]
ideal_weather_df

,Unnamed: 0,city,lat,lon,max temp,humidity,cloudiness,wind speed,country,date
40,40,Filingué,14.3521,3.3168,82.26,10,1,8.43,NE,1644342985
72,72,Posadas,-27.3671,-55.8961,87.78,30,0,9.22,AR,1644342997
86,86,Pinar del Río,22.4175,-83.6981,83.48,47,0,9.37,CU,1644343002
112,112,Lokoja,7.8024,6.7430,89.51,14,45,4.63,NG,1644343010
152,152,Ybycuí,-26.0167,-57.0500,91.58,21,0,6.85,PY,1644343025
187,187,Conceição do Araguaia,-8.2578,-49.2647,91.24,52,38,2.86,BR,1644343038
223,223,San Juan,-31.5375,-68.5364,91.60,21,0,6.13,AR,1644343034
228,228,Jardim,-21.4803,-56.1381,92.89,14,0,8.10,BR,1644343052
240,240,Sokolo,14.7328,-6.1219,85.35,8,0,9.44,ML,1644343056


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# from 6-APIs, class 3, 05-Stu_Google_Complex
hotel_df = pd.DataFrame(ideal_weather_df, columns=['city', 'country', 'lat', 'lon'])

hotel_df['hotel name'] = ""
hotel_df

,city,country,lat,lon,hotel name
40,Filingué,NE,14.3521,3.3168,
72,Posadas,AR,-27.3671,-55.8961,
86,Pinar del Río,CU,22.4175,-83.6981,
112,Lokoja,NG,7.8024,6.7430,
152,Ybycuí,PY,-26.0167,-57.0500,
187,Conceição do Araguaia,BR,-8.2578,-49.2647,
223,San Juan,AR,-31.5375,-68.5364,
228,Jardim,BR,-21.4803,-56.1381,
240,Sokolo,ML,14.7328,-6.1219,


In [9]:
#copied directly from 6-APIs, class 3, 05-Stu_Google_Complex

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    print(f"retrieving results for {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    results = response['results']

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        print(f"the closest hotel is {results [0] ['name']}.")
        hotel_df.loc[index, "hotel name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("--------------")

retrieving results for 40: Filingué.
Missing field/result... skipping.
--------------
retrieving results for 72: Posadas.
the closest hotel is Hotel Posadas.
--------------
retrieving results for 86: Pinar del Río.
the closest hotel is Hostal La Colina Dunay y Raul.
--------------
retrieving results for 112: Lokoja.
the closest hotel is Hotel Charvid.
--------------
retrieving results for 152: Ybycuí.
the closest hotel is Villa América.
--------------
retrieving results for 187: Conceição do Araguaia.
the closest hotel is Tarumã Tropical Hotel.
--------------
retrieving results for 223: San Juan.
the closest hotel is Hotel Provincial.
--------------
retrieving results for 228: Jardim.
the closest hotel is Estancia Hotel - Jardim MS.
--------------
retrieving results for 240: Sokolo.
Missing field/result... skipping.
--------------


In [10]:
hotel_df

,city,country,lat,lon,hotel name
40,Filingué,NE,14.3521,3.3168,
72,Posadas,AR,-27.3671,-55.8961,Hotel Posadas
86,Pinar del Río,CU,22.4175,-83.6981,Hostal La Colina Dunay y Raul
112,Lokoja,NG,7.8024,6.7430,Hotel Charvid
152,Ybycuí,PY,-26.0167,-57.0500,Villa América
187,Conceição do Araguaia,BR,-8.2578,-49.2647,Tarumã Tropical Hotel
223,San Juan,AR,-31.5375,-68.5364,Hotel Provincial
228,Jardim,BR,-21.4803,-56.1381,Estancia Hotel - Jardim MS
240,Sokolo,ML,14.7328,-6.1219,


In [101]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [112]:
# This code works to add the markers, but they do not show the template information when they're clicked on
hotel_layer = gmaps.symbol_layer(
            locations, fill_color='rgba(0, 150, 0, 0.4)',
            stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
            info_box_content = (hotel_info))

fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(markers)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))